```
This notebook sets up and runs a test case for analyzing Kelvin waves
Copyright (C) 2018 - 2022 SINTEF Digital
Copyright (C) 2018 - 2022 Norwegian Meteorological Institute

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [1]:
import sys
gpuocean_path = [p[:-4] for p in sys.path if (p.endswith("gpuocean/src") or p.endswith("gpuocean\\src"))][0]
import git
repo = git.Repo(gpuocean_path)
print("GPUOcean code from:", repo.head.object.hexsha, "on branch", repo.active_branch.name)

GPUOcean code from: 0bff8670195d21d3f7f0df772874a6ca770b6fe0 on branch havahol_reduced_gravity


# Oslofjord
Testing of Nils projected files

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
from netCDF4 import Dataset
import datetime, copy
from IPython.display import display

#For plotting
import matplotlib
from matplotlib import pyplot as plt

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

In [ ]:
from gpuocean.utils import IPythonMagic, Common, NetCDFInitialization, WindStress, OceanographicUtilities

from gpuocean.SWEsimulators import CDKLM16

In [ ]:
%cuda_context_handler barotropic_gpu_ctx

In [ ]:
%cuda_context_handler baroclinic_gpu_ctx

Path to the test file

In [ ]:
source_url = "/sintef/data/OsloFjord/test_polstere_1h_0007.nc"

## Inspecting file structure and content

In [ ]:
from netCDF4 import Dataset
nc = Dataset(source_url)

In [ ]:
import xarray as xr
ds = xr.open_dataset(source_url)
ds

Animation utils

In [ ]:
import animation_utils

## Generating GPUOcean Simulation from Input

General parameters

In [ ]:
dimY, dimX = ds.h.data.shape

In [ ]:
x0, x1, y0, y1 = 5, dimX-5, 175, dimY-5

Simulation span: 6h!

In [ ]:
t_start = 5*24
t_stop =  5*24 + 6

In [ ]:
T = (t_stop-t_start)*3600  #Input

In [ ]:
timestep_indices = [list(np.arange(t_start, t_stop+1))]

Generating wind fields

In [ ]:
from gpuocean.utils import WindStress

def generate_wind_field(wind_angle_deg, T, shift=None):
    """Generating WindStress object with spatially constant, hourly varying wind 
    according to the standard shape in Oslofjord (see Oslofjord-ArtificialParameters.ipynb)
    
    wind_angle_deg - angle of wind field towards north in degree!
    """

    wind_angle = np.deg2rad(wind_angle_deg)

    # Reference wind speed for 0 - 24h 
    wind_u_ref = np.zeros(24)
    wind_u_ref[:10] = -0.5*(np.sin(2*np.pi*(np.arange(0,10)-2.5)/10)+1)
    wind_u_ref[10:] = 5*(np.sin(2*np.pi*(np.arange(10,24)+0.5)/14)+1)

    # Wind fields for 0 - 24h
    wind_u = list(np.cos(wind_angle)*wind_u_ref[:,np.newaxis][:,np.newaxis].astype(np.float32))
    wind_v = list(np.sin(wind_angle)*wind_u_ref[:,np.newaxis][:,np.newaxis].astype(np.float32))

    # Shifting that reference hours align with t_0
    if shift is None:
        shift = datetime.datetime.utcfromtimestamp(nc["ocean_time"][t_start]).hour
    for shift_hour in range(shift):
        wind_u.append(wind_u.pop(0))
        wind_v.append(wind_v.pop(0))

    # Cut relevant time span
    wind_u = wind_u[0:int(T/3600+1)]
    wind_v = wind_v[0:int(T/3600 +1)]

    # Construct time array in simulator time (starting with 0)
    ts = np.arange(T+1, step=3600)

    # Init WindStress object
    wind = WindStress.WindStress(t=ts, wind_u=wind_v, wind_v=wind_u)
    wind.compute_wind_stress_from_wind()

    return wind 

### Initial conditions

In [ ]:
barotropic_data_args, baroclinic_data_args = NetCDFInitialization.getCombinedInitialConditions(source_url, x0, x1, y0, y1, 1024.0, timestep_indices=timestep_indices, norkyst_data=False, land_value=0.0, download_data=False)

#### Barotropic Simulations

In [ ]:
barotropic_data_args["wind"] = generate_wind_field(45.0, T)

In [ ]:
barotropic_sim = CDKLM16.CDKLM16(barotropic_gpu_ctx, **NetCDFInitialization.removeMetadata(barotropic_data_args), dt=0.0, write_netcdf=True)

In [ ]:
bt_eta, bt_hu, bt_hv = barotropic_sim.download()

fig, axs = plt.subplots(1,3, figsize=(15,5))

im = axs[0].imshow(bt_eta, vmin=-0.5, vmax=0.5, cmap="coolwarm")
axs[0].set_title("eta")
plt.colorbar(im, ax=axs[0], shrink=0.5)

im = axs[1].imshow(bt_hu, vmin=-5, vmax=5, cmap="coolwarm")
axs[1].set_title("hu")
plt.colorbar(im, ax=axs[1], shrink=0.5)

im = axs[2].imshow(bt_hv, vmin=-5, vmax=5, cmap="coolwarm")
axs[2].set_title("hv")
plt.colorbar(im, ax=axs[2], shrink=0.5)

In [ ]:
subt = 3600
for runt in range(int(T/subt)):
    barotropic_sim.step(subt)


In [ ]:
with plt.rc_context({'lines.color':'black', 
                        'text.color':'black', 
                        'axes.labelcolor':'black', 
                        'xtick.color':'black',
                        'ytick.color':'black'}):
    anim = animation_utils.ncAnimation(barotropic_sim.sim_writer.output_file_name, "gpuocean", comment="GPUOcean")
    anim.save("anims/barotropic_test.mp4")

#### Baroclinic Simulations

In [ ]:
baroclinic_data_args["wind"] = generate_wind_field(45.0, T)
baroclinic_data_args["wind_stress_factor"] = 0.3

In [ ]:
baroclinic_sim = CDKLM16.CDKLM16(baroclinic_gpu_ctx, **NetCDFInitialization.removeMetadata(baroclinic_data_args), dt=0.0, write_netcdf=True)

In [ ]:
bc_eta, bc_hu, bc_hv = baroclinic_sim.download()

fig, axs = plt.subplots(1,3, figsize=(15,5))

im = axs[0].imshow(-bc_eta, vmin=-15, vmax=0, cmap="coolwarm")
axs[0].set_title("MLD")
plt.colorbar(im, ax=axs[0], shrink=0.5)

im = axs[1].imshow(bc_hu, vmin=-5, vmax=5, cmap="coolwarm")
axs[1].set_title("hu")
plt.colorbar(im, ax=axs[1], shrink=0.5)

im = axs[2].imshow(bc_hv, vmin=-5, vmax=5, cmap="coolwarm")
axs[2].set_title("hv")
plt.colorbar(im, ax=axs[2], shrink=0.5)

In [ ]:
subt = 3600
for runt in range(int(T/subt)):
    baroclinic_sim.step(subt)

In [ ]:
with plt.rc_context({'lines.color':'black', 
                        'text.color':'black', 
                        'axes.labelcolor':'black', 
                        'xtick.color':'black',
                        'ytick.color':'black'}):
    anim = animation_utils.ncAnimation(baroclinic_sim.sim_writer.output_file_name, "gpuocean-reduced_grav", comment="GPUOcean")
    anim.save("anims/baroclinic_test.mp4")

#### Combined deterministic simulation with drifters

Just for reference

In [ ]:
from gpuocean.SWEsimulators import CombinedCDKLM16

sims = CombinedCDKLM16.CombinedCDKLM16(barotropic_sim=barotropic_sim, baroclinic_sim=baroclinic_sim)

In [ ]:
from gpuocean.utils import Observation
from gpuocean.drifters import GPUDrifterCollection
from gpuocean.dataassimilation import DataAssimilationUtils as dautils

In [ ]:
observation_type = dautils.ObservationType.UnderlyingFlow 
    
observation_args = {'observation_type': observation_type,
                'nx': sims.nx, 'ny': sims.ny,
                'domain_size_x': sims.nx*sims.dx,
                'domain_size_y': sims.ny*sims.dy,
                'land_mask': sims.getLandMask()
                }

trajectories = Observation.Observation(**observation_args)

In [ ]:
initx = [ 6000]
inity = [12000]

num_drifters = len(initx)

drifters = GPUDrifterCollection.GPUDrifterCollection(sims.barotropic_gpu_ctx, num_drifters,
                                                    boundaryConditions = sims.boundary_conditions,
                                                    domain_size_x = trajectories.domain_size_x,
                                                    domain_size_y = trajectories.domain_size_y,
                                                    gpu_stream = sims.gpu_stream,
                                                    wind=barotropic_data_args["wind"], # using the barotropic wind
                                                    wind_drift_factor=0.02/2 # halfend since drift kernel called twice
                                                    )

drifter_pos_init = np.array([initx, inity]).T
drifters.setDrifterPositions(drifter_pos_init)

In [ ]:
sims.attachDrifters(drifters)
trajectories.add_observation_from_sim(sims)

In [ ]:
sims.combinedStep(T, trajectory_dt=60, trajectories=trajectories)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))

combined_eta, combined_u, combined_v = sims.download() 
uv_field = np.sqrt(combined_u**2 + combined_v**2)


domain_extent = [0, sims.nx*sims.dx, 0, sims.ny*sims.dy]

velo_cmap = copy.deepcopy(plt.cm.Reds)
velo_cmap.set_bad("grey", alpha = 1.0)

ax.imshow(uv_field, interpolation="none", origin='lower', 
                cmap=velo_cmap, 
                vmin=0.0, vmax=0.5, 
                extent=domain_extent)

drifter_path = trajectories.get_drifter_path(0, 0, sims.t, in_km = False)[0]

start_pos = drifter_path[0,:]
circ_start = matplotlib.patches.Circle((start_pos[0], start_pos[1]), 
                                                100, color = 'blue',
                                                fill=True, zorder=10)
ax.add_patch(circ_start)

ax.plot(drifter_path[:,0], drifter_path[:,1], color="blue", ls="-", zorder=5, label="Combined")
end_pos = drifter_path[-1,:]
ax.plot(end_pos[0], end_pos[1], 'x', color='k', zorder=11)


ax.legend(labelcolor="black")


## Cross Pert Drifters


The test scenario should be
- 2 barotropic simulations (different `wind`)
- 3 wind drift factors (connected to barotropic sims, since wind response of baroclinic model is already well covered)
- 12 baroclinic simulations (2 different `wind`, with 2 different `wind_stress_factors` and 3 different `g`)

The wind information is shared between different ensemble members

In [ ]:
import pandas as pd 

In [ ]:
wind_angles_degs = [0.0, 22.5, 45.0]

winds = []
for wind_angle_deg in wind_angles_degs:
    winds.append( generate_wind_field(wind_angle_deg, T) )

In [ ]:
bt_gpu_ctxs = []
bc_gpu_ctxs = []
for i in range(len(wind_angles_degs)):
    bt_gpu_ctxs.append( Common.CUDAContext() )
    bc_gpu_ctxs.append( Common.CUDAContext() )

We need contextes per wind field per simulation type

In [ ]:
barotropic_sims = []

for i in range(2):
    barotropic_data_args["wind"] = winds[i]
    barotropic_sims.append( CDKLM16.CDKLM16(bt_gpu_ctxs[i], **NetCDFInitialization.removeMetadata(barotropic_data_args),  dt=0.0))

In [ ]:
wind_stress_factors = [0.3, 0.5]
gs = [0.01, baroclinic_sim.g, 0.1]

baroclinic_sims = []
bc_table = pd.DataFrame(columns=["baroclinic_id", "wind_angle_id", "wind_stress_factor_id", "g_id"]).set_index("baroclinic_id")

for i_w in range(len(wind_angles_degs)):
    for i_wsf in range(len(wind_stress_factors)):
        for i_g in range(len(gs)):
            baroclinic_data_args["wind"] = winds[i_w]
            baroclinic_data_args["wind_stress_factor"] = wind_stress_factors[i_wsf]
            baroclinic_data_args["g"] = gs[i_g]
            baroclinic_sims.append( CDKLM16.CDKLM16(bc_gpu_ctxs[i_w], **NetCDFInitialization.removeMetadata(baroclinic_data_args),  dt=0.0))
            bc_table.loc[len(bc_table.index)] = [i_w, i_wsf, i_g]

In [ ]:
bc_table

In [ ]:
wind_drift_factors = [0.01, 0.02, 0.03]

Thats what we have:

In [ ]:
ref_table = pd.DataFrame(columns=["drifter_id", "barotropic_id", "baroclinic_id", "wind_drift_factor_id"]).set_index("drifter_id")

In [ ]:
for bt in range(len(barotropic_sims)):
    for bc in range(len(baroclinic_sims)):
        for wind in range(len(wind_drift_factors)):
            ref_table.loc[len(ref_table.index)] = [bt, bc, wind]

In [ ]:
ref_table

Collecting drifter and observation objects

In [ ]:
from gpuocean.utils import Observation
from gpuocean.drifters import GPUDrifterCollection
from gpuocean.dataassimilation import DataAssimilationUtils as dautils

In [ ]:
observation_type = dautils.ObservationType.UnderlyingFlow 
    
observation_args = {'observation_type': observation_type,
                'nx': barotropic_sim.nx, 'ny': barotropic_sim.ny,
                'domain_size_x': barotropic_sim.nx*barotropic_sim.dx,
                'domain_size_y': barotropic_sim.ny*barotropic_sim.dy,
                'land_mask': barotropic_sim.getLandMask()
                }

trajectories = Observation.Observation(**observation_args)


In [ ]:
initx = [ 6000]
inity = [12000]

num_drifters = len(initx)

In [ ]:
crossprod_trajectories = []
for cp in range(len(ref_table)):
    crossprod_trajectories.append(copy.deepcopy(trajectories))

In [ ]:
crossprod_drifters = []
for cp in range(len(ref_table)): 
    drifters = GPUDrifterCollection.GPUDrifterCollection(barotropic_sims[ref_table.iloc[cp].barotropic_id].gpu_ctx, # OBS: This is used for wind drift! 
                                                    num_drifters,
                                                    boundaryConditions = barotropic_sim.boundary_conditions,
                                                    domain_size_x = trajectories.domain_size_x,
                                                    domain_size_y = trajectories.domain_size_y,
                                                    gpu_stream = barotropic_sim.gpu_stream,
                                                    wind=winds[ref_table.iloc[cp].barotropic_id], # using the barotropic wind
                                                    wind_drift_factor=wind_drift_factors[ref_table.iloc[cp].wind_drift_factor_id]/2 # drift is called twice per step, but we only want one wind contribution
                                                    )           

    drifter_pos_init = np.array([initx, inity]).T
    drifters.setDrifterPositions(drifter_pos_init)
    crossprod_drifters.append(drifters)

Attach CPdrifters

In [ ]:
from itertools import compress

In [ ]:
for bt in range(len(barotropic_sims)):
    barotropic_sims[bt].attachCrossProductDrifters( list(compress(crossprod_drifters, ref_table["barotropic_id"] == bt)), [baroclinic_sims[i] for i in list(ref_table[ref_table["barotropic_id"] == bt].baroclinic_id)] )

Let's fan out the drifters!

In [ ]:
bc_dt = min([bc.dt for bc in baroclinic_sims])

In [ ]:
for bc in baroclinic_sims:
    bc.step(bc_dt)

In [ ]:
for bt in barotropic_sims:
    bt.step(bc_dt)

In [ ]:
for d in range(len(crossprod_drifters)):
    crossprod_trajectories[d].add_observation_from_drifters(crossprod_drifters[d], bt.t)

In [ ]:
while bt.t < T:

    bc_dt = min([bc.dt for bc in baroclinic_sims])
    if bt.t % 3600 != 0:
        bc_dt = min(bc_dt, np.ceil(bt.t/3600)*3600 - bt.t)

    for bc in baroclinic_sims:
        bc.step(bc_dt)

    for bt in barotropic_sims:
        bt.step(bc_dt)

    for d in range(len(crossprod_drifters)):
        crossprod_trajectories[d].add_observation_from_drifters(crossprod_drifters[d], bt.t)

In [ ]:
with plt.rc_context({'lines.color':'black', 
                        'text.color':'black', 
                        'axes.labelcolor':'black', 
                        'xtick.color':'black',
                        'ytick.color':'black'}):
    fig, ax = plt.subplots(1,1, figsize=(7,7))
    ax.tick_params(axis='both', which='major', labelsize=18)
    domain_extent = [0, sims.nx*sims.dx/1000, 0, sims.ny*sims.dy/1000]

    bg_cmap = copy.deepcopy(plt.cm.Blues)
    bg_cmap.set_bad("grey", alpha = 1.0)

    ax.imshow(barotropic_sims[0].download()[0] > -10, interpolation="none", origin='lower', 
                    cmap=bg_cmap,  
                    extent=domain_extent)

    for path in [t.get_drifter_path(0, 0, T, in_km = True)[0] for t in crossprod_trajectories]:
        ax.plot(path[:,0], path[:,1], color="blue", ls="-", zorder=5, alpha=0.1)

    plt.tight_layout()

    plt.savefig("figs/CPdrift.pdf")

### Purely barotropic 

In [ ]:
pure_barotropic_sims = []

for i in range(2):
    barotropic_data_args["wind"] = winds[i]
    pure_barotropic_sims.append( CDKLM16.CDKLM16(bt_gpu_ctxs[i], **NetCDFInitialization.removeMetadata(barotropic_data_args),  dt=0.0))

In [ ]:
observation_type = dautils.ObservationType.UnderlyingFlow 
    
observation_args = {'observation_type': observation_type,
                'nx': barotropic_sim.nx, 'ny': barotropic_sim.ny,
                'domain_size_x': barotropic_sim.nx*barotropic_sim.dx,
                'domain_size_y': barotropic_sim.ny*barotropic_sim.dy,
                'land_mask': barotropic_sim.getLandMask()
                }

trajectories = Observation.Observation(**observation_args)

In [ ]:
initx = [ 6000]
inity = [12000]

num_drifters = len(initx)

In [ ]:
import pandas as pd
bt_table = pd.DataFrame(columns=["drifter_id", "barotropic_id", "wind_drift_factor_id"]).set_index("drifter_id")

for bt in range(len(pure_barotropic_sims)):
    for wind in range(len(wind_drift_factors)):
        bt_table.loc[len(bt_table.index)] = [bt, wind]

In [ ]:
bt_table

In [ ]:
crossbt_trajectories = []
for cp in range(len(bt_table)):
    crossbt_trajectories.append(copy.deepcopy(trajectories))

In [ ]:
crossbt_drifters = []
for cp in range(len(bt_table)): 
    drifters = GPUDrifterCollection.GPUDrifterCollection(pure_barotropic_sims[bt_table.iloc[cp].barotropic_id].gpu_ctx, # OBS: This is used for wind drift! 
                                                    num_drifters,
                                                    boundaryConditions = barotropic_sim.boundary_conditions,
                                                    domain_size_x = trajectories.domain_size_x,
                                                    domain_size_y = trajectories.domain_size_y,
                                                    gpu_stream = barotropic_sim.gpu_stream,
                                                    wind=winds[bt_table.iloc[cp].barotropic_id], # using the barotropic wind
                                                    wind_drift_factor=wind_drift_factors[bt_table.iloc[cp].wind_drift_factor_id])          

    drifter_pos_init = np.array([initx, inity]).T
    drifters.setDrifterPositions(drifter_pos_init)
    crossbt_drifters.append(drifters)

In [ ]:
from itertools import compress

for bt in range(len(pure_barotropic_sims)):
    pure_barotropic_sims[bt].attachCrossProductDrifters( list(compress(crossbt_drifters, bt_table["barotropic_id"] == bt)), int(np.sum(bt_table["barotropic_id"] == bt))* [None] )

In [ ]:
while pure_barotropic_sims[0].t < T:
    for bt in pure_barotropic_sims:
        bt.step(60)

    for d in range(len(crossbt_drifters)):
        crossbt_trajectories[d].add_observation_from_drifters(crossbt_drifters[d], bt.t)

In [ ]:
with plt.rc_context({'lines.color':'black', 
                        'text.color':'black', 
                        'axes.labelcolor':'black', 
                        'xtick.color':'black',
                        'ytick.color':'black'}):
    fig, ax = plt.subplots(1,1, figsize=(7,7))
    ax.tick_params(axis='both', which='major', labelsize=18)
    domain_extent = [0, sims.nx*sims.dx/1000, 0, sims.ny*sims.dy/1000]

    bg_cmap = copy.deepcopy(plt.cm.Blues)
    bg_cmap.set_bad("grey", alpha = 1.0)

    ax.imshow(pure_barotropic_sims[0].download()[0] > -10, interpolation="none", origin='lower', 
                    cmap=bg_cmap,  
                    extent=domain_extent)

    for path in [t.get_drifter_path(0, 0, T, in_km = True)[0] for t in crossbt_trajectories]:
        ax.plot(path[:,0], path[:,1], color="blue", ls="-", zorder=5, alpha=0.1)

    plt.tight_layout()

    plt.savefig("figs/CPdrift-bt.pdf")